In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt 
import seaborn as sns

#폰트 지정
import matplotlib.font_manager as fm
font_name=fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family=font_name)

mpl.rcParams["axes.unicode_minus"] = False

---
## 서울시 코로나바이러스 감염자 추세 확인

+ 부제1 : 언제 가장 많은 확진자가 발생하였는가?
+ 부제2 : 어느 구에서 가장 많은 확진자가 발생하였는가?

### 1. 데이터 수집

+ 공공데이터포털(https://www.data.go.kr)에 회원가입 후 파일 내려받기
+ 데이터1 : '서울시 코로나19 확진자 현황.csv' 
+ 데이터2 : '보건복지부_코로나19 연령별·성별감염_현황.xml'
+ 데이터3 : '서울시 주민등록인구 (동별)통계.txt'

### 2. 데이터1 전처리
+ 사용할 컬럼만 고르기(확진일, 지역(구), 여행력, 접촉력, 상태)
+ 여행력 NaN는 '국내'로 변경
+ 상태 NaN는 '확인중'으로 변경
+ 접촉력은 상위 10개만 도출하여 그래프

In [2]:
### 데이터1 : 12월 31일 데이터
seoul_corona = pd.read_csv("data/seoul/서울시 코로나19 확진자 현황.csv", encoding="euc-kr", thousands=",")
seoul_corona.tail(10)

,연번,확진일,환자번호,국적,환자정보,지역,여행력,접촉력,조치사항,상태,이동경로,등록일,수정일,노출여부
19353,10,20.2.5.,NaN,NaN,NaN,성북구,NaN,#6 접촉,NaN,퇴원,이동경로 공개기간 경과,2020-12-31 09:16:12,2020-12-31 09:16:12,Y
19354,9,20.2.5.,NaN,NaN,NaN,송파구,싱가포르,해외 접촉,NaN,퇴원,이동경로 공개기간 경과,2020-12-31 09:16:12,2020-12-31 09:16:12,Y
19355,8,20.2.2.,NaN,NaN,NaN,타시도,우한 교민,해외 접촉,NaN,퇴원,이동경로 공개기간 경과,2020-12-31 09:16:12,2020-12-31 09:16:12,Y
19356,7,20.1.31.,NaN,NaN,NaN,종로구,NaN,#6 접촉,NaN,퇴원,이동경로 공개기간 경과,2020-12-31 09:16:12,2020-12-31 09:16:12,Y
19357,6,20.1.31.,NaN,NaN,NaN,종로구,NaN,#6 접촉,NaN,퇴원,이동경로 공개기간 경과,2020-12-31 09:16:12,2020-12-31 09:16:12,Y
19358,5,20.1.31.,NaN,NaN,NaN,성북구,NaN,#5 접촉,NaN,퇴원,이동경로 공개기간 경과,2020-12-31 09:16:12,2020-12-31 09:16:12,Y
19359,4,20.1.30.,NaN,NaN,NaN,마포구,중국 우한시,해외 접촉,NaN,퇴원,이동경로 공개기간 경과,2020-12-31 09:16:12,2020-12-31 09:16:12,Y
19360,3,20.1.30.,NaN,NaN,NaN,종로구,NaN,#3 접촉,NaN,퇴원,이동경로 공개기간 경과,2020-12-31 09:16:12,2020-12-31 09:16:12,Y
19361,2,20.1.30.,NaN,NaN,NaN,중랑구,중국 우한시,해외 접촉,NaN,퇴원,이동경로 공개기간 경과,2020-12-31 09:16:12,2020-12-31 09:16:12,Y
19362,1,20.1.24.,NaN,NaN,NaN,강서구,중국 우한시,해외 접촉,NaN,퇴원,이동경로 공개기간 경과,2020-12-31 09:16:12,2020-12-31 09:16:12,Y


#### 2-1. 기본 전처리

In [3]:
# 사용할 컬럼만 고르기(확진일, 지역(구), 여행력, 접촉력, 상태)
seoul_corona2 = seoul_corona[["확진일", "지역", "여행력", "접촉력", "상태"]]
seoul_corona2.tail(10)
#print(type(seoul_corona2))

,확진일,지역,여행력,접촉력,상태
19353,20.2.5.,성북구,NaN,#6 접촉,퇴원
19354,20.2.5.,송파구,싱가포르,해외 접촉,퇴원
19355,20.2.2.,타시도,우한 교민,해외 접촉,퇴원
19356,20.1.31.,종로구,NaN,#6 접촉,퇴원
19357,20.1.31.,종로구,NaN,#6 접촉,퇴원
19358,20.1.31.,성북구,NaN,#5 접촉,퇴원
19359,20.1.30.,마포구,중국 우한시,해외 접촉,퇴원
19360,20.1.30.,종로구,NaN,#3 접촉,퇴원
19361,20.1.30.,중랑구,중국 우한시,해외 접촉,퇴원
19362,20.1.24.,강서구,중국 우한시,해외 접촉,퇴원


In [4]:
# 여행력 NaN은 '국내감염'으로, 상태 NaN은 '미정'으로 변경
missing_fill_val = {"여행력":"국내감염", "상태":"미정"}
seoul_corona2 = seoul_corona2.fillna(missing_fill_val)

seoul_corona2

,확진일,지역,여행력,접촉력,상태
0,20.12.31.,강북구,국내감염,감염경로 조사중,미정
1,20.12.31.,양천구,국내감염,기타 확진자 접촉,미정
2,20.12.31.,도봉구,국내감염,기타 확진자 접촉,미정
3,20.12.31.,중랑구,국내감염,기타 확진자 접촉,미정
4,20.12.31.,강북구,국내감염,감염경로 조사중,미정
...,...,...,...,...,...
19358,20.1.31.,성북구,국내감염,#5 접촉,퇴원
19359,20.1.30.,마포구,중국 우한시,해외 접촉,퇴원
19360,20.1.30.,종로구,국내감염,#3 접촉,퇴원
19361,20.1.30.,중랑구,중국 우한시,해외 접촉,퇴원


In [5]:
# 지역 데이터의 타지역 표기 및 오타 등을 확인
#seoul_corona2["지역"].isnull()
seoul_corona2["지역"].unique()

array(['강북구', '양천구', '도봉구', '중랑구', '광진구', '기타', '은평구 ', '은평구', '용산구',
       '중구 ', '송파구', '마포구', '동작구', '중구', '노원구', '강서구', '강남구', '영등포구',
       '서초구', '관악구', '금천구', '동대문구', '성동구', '종로구', '타시도', '강동구', '서대문구',
       '성북구', '구로구', '은평구   ', '동작구 ', '송파구 ', '경기도 의왕시', '마포구 ', '앙쳔구',
       '경기도 광명시', '경기도 하남시', nan, '부산', '인천시', '동대문', '영등포', '경기도', '경기',
       '부천시', '도붕구', '경기도 성남시', '고양시', '경기도 시흥시', '강원도', '중랑구 면목동', '중로구',
       '인천', '제주시', '은평', '증링구'], dtype=object)

In [6]:
# 오타 수정
seoul_corona2 = seoul_corona2.replace({"지역":"동대문"},{"지역":"동대문구"})
seoul_corona2 = seoul_corona2.replace({"지역":"영등포"},{"지역":"영등포구"})
seoul_corona2 = seoul_corona2.replace({"지역":"도붕구"},{"지역":"도봉구"})
seoul_corona2 = seoul_corona2.replace({"지역":"중랑구 면목동"},{"지역":"중랑구"})
seoul_corona2 = seoul_corona2.replace({"지역":"중로구"},{"지역":"종로구"})
seoul_corona2 = seoul_corona2.replace({"지역":"은평"},{"지역":"은평구"})
seoul_corona2 = seoul_corona2.replace({"지역":"증링구"},{"지역":"중랑구"})

In [96]:
seoul_corona2["지역"].unique()

array(['강북구', '양천구', '도봉구', '중랑구', '광진구', '기타', '은평구 ', '은평구', '용산구',
       '중구 ', '송파구', '마포구', '동작구', '중구', '노원구', '강서구', '강남구', '영등포구',
       '서초구', '관악구', '금천구', '동대문구', '성동구', '종로구', '타시도', '강동구', '서대문구',
       '성북구', '구로구', '은평구   ', '동작구 ', '송파구 ', '경기도 의왕시', '마포구 ', '앙쳔구',
       '경기도 광명시', '경기도 하남시', '부산', '인천시', '경기도', '경기', '부천시', '경기도 성남시',
       '고양시', '경기도 시흥시', '강원도', '인천', '제주시'], dtype=object)

In [7]:
# nan 값 대체
seoul_corona2 = seoul_corona2.fillna({"지역":"기타"})

In [8]:
# 타시도 값 대체
seoul_corona2 = seoul_corona2.replace({"지역": "경기도 의왕시"}, {"지역":"타시도"}) 
seoul_corona2 = seoul_corona2.replace({"지역": "경기도 광명시"}, {"지역":"타시도"})  
seoul_corona2 = seoul_corona2.replace({"지역": "경기도 하남시"}, {"지역":"타시도"})
seoul_corona2 = seoul_corona2.replace({"지역": "부산"}, {"지역":"타시도"})
seoul_corona2 = seoul_corona2.replace({"지역": "인천시"}, {"지역":"타시도"})
seoul_corona2 = seoul_corona2.replace({"지역": "경기도"}, {"지역":"타시도"})
seoul_corona2 = seoul_corona2.replace({"지역": "경기"}, {"지역":"타시도"})
seoul_corona2 = seoul_corona2.replace({"지역": "부천시"}, {"지역":"타시도"})
seoul_corona2 = seoul_corona2.replace({"지역": "경기도 성남시"}, {"지역":"타시도"})
seoul_corona2 = seoul_corona2.replace({"지역": "고양시"}, {"지역":"타시도"})
seoul_corona2 = seoul_corona2.replace({"지역": "경기도 시흥시"}, {"지역":"타시도"})
seoul_corona2 = seoul_corona2.replace({"지역": "강원도"}, {"지역":"타시도"})
seoul_corona2 = seoul_corona2.replace({"지역": "인천"}, {"지역":"타시도"})
seoul_corona2 = seoul_corona2.replace({"지역": "제주시"}, {"지역":"타시도"})

In [9]:
seoul_corona2["지역"].unique()

array(['강북구', '양천구', '도봉구', '중랑구', '광진구', '기타', '은평구 ', '은평구', '용산구',
       '중구 ', '송파구', '마포구', '동작구', '중구', '노원구', '강서구', '강남구', '영등포구',
       '서초구', '관악구', '금천구', '동대문구', '성동구', '종로구', '타시도', '강동구', '서대문구',
       '성북구', '구로구', '은평구   ', '동작구 ', '송파구 ', '마포구 ', '앙쳔구'],
      dtype=object)

In [10]:
# 컬럼 순서변경
#seoul_corona2 = seoul_corona2[["지역", "확진일", "상태", "여행력", "접촉력"]]
seoul_corona2.loc[:, ["지역", "확진일", "상태", "여행력", "접촉력"]]
seoul_corona2.head(10)
seoul_corona2

,확진일,지역,여행력,접촉력,상태
0,20.12.31.,강북구,국내감염,감염경로 조사중,미정
1,20.12.31.,양천구,국내감염,기타 확진자 접촉,미정
2,20.12.31.,도봉구,국내감염,기타 확진자 접촉,미정
3,20.12.31.,중랑구,국내감염,기타 확진자 접촉,미정
4,20.12.31.,강북구,국내감염,감염경로 조사중,미정
...,...,...,...,...,...
19358,20.1.31.,성북구,국내감염,#5 접촉,퇴원
19359,20.1.30.,마포구,중국 우한시,해외 접촉,퇴원
19360,20.1.30.,종로구,국내감염,#3 접촉,퇴원
19361,20.1.30.,중랑구,중국 우한시,해외 접촉,퇴원


##### 2-3. 지역별로 구분
+ 지역 기준으로 변경(index = 지역)
+ 월별로 나눈 데이터를 구별로 세분화하여 그래프
+ 서울시 지도를 이용하여 시각화

In [33]:
# 확진자수 컬럼 추가 
seoul_corona2["확진자 수"] = 1
seoul_corona2.head(30)
#print(type(seoul_corona2))

             확진일   지역     여행력        접촉력  상태  확진자 수
0      20.12.31.  강북구    국내감염   감염경로 조사중  미정      1
1      20.12.31.  양천구    국내감염  기타 확진자 접촉  미정      1
2      20.12.31.  도봉구    국내감염  기타 확진자 접촉  미정      1
3      20.12.31.  중랑구    국내감염  기타 확진자 접촉  미정      1
4      20.12.31.  강북구    국내감염   감염경로 조사중  미정      1
...          ...  ...     ...        ...  ..    ...
19358   20.1.31.  성북구    국내감염      #5 접촉  퇴원      1
19359   20.1.30.  마포구  중국 우한시      해외 접촉  퇴원      1
19360   20.1.30.  종로구    국내감염      #3 접촉  퇴원      1
19361   20.1.30.  중랑구  중국 우한시      해외 접촉  퇴원      1
19362   20.1.24.  강서구  중국 우한시      해외 접촉  퇴원      1

[19363 rows x 6 columns]


AttributeError: 'NoneType' object has no attribute 'header'

In [28]:
#seoul_corona2.merge(seoul_corona2)
#seoul_corona2.merge("지역", "확진일")
#seoul_corona2(["확진자수"]).sum()
#seoul_corona3 = seoul_corona2.filter(["지역", "확진일"]).agg(["확진자수"])
#seoul_corona3 = seoul_corona2.groupby({"확진일"})#.agg({"확진자수":["count"]})
#seoul_corona3.head(50)

AttributeError: '확진자수' is not a valid function for 'Series' object

In [39]:
seoul_corona3 = seoul_corona2.groupby(seoul_corona2["지역"], seoul_corona2["확진일"], as_index=False).sum()
seoul_corona3.columns = ["지역", "확진일"]
seoul_corona3

KeyError: '확진자수'

In [42]:
# 
seoul_corona3 = seoul_corona2.pivot("지역", "확진일")
seoul_corona3

ValueError: Index contains duplicate entries, cannot reshape

In [53]:
# 각 구별 확진자 수 컬럼 추가
seoul_corona2["일일 확진자수"] = seoul_corona2["지역"] + seoul_corona2["확진일"]
seoul_corona2

seoul_corona2["일일 확진자수"] = seoul_corona2[""].sum(seoul_corona2["확진자수"])

,확진일,지역,여행력,접촉력,상태,확진자 수,일일 확진자수
0,20.12.31.,강북구,국내감염,감염경로 조사중,미정,1,강북구20.12.31.
1,20.12.31.,양천구,국내감염,기타 확진자 접촉,미정,1,양천구20.12.31.
2,20.12.31.,도봉구,국내감염,기타 확진자 접촉,미정,1,도봉구20.12.31.
3,20.12.31.,중랑구,국내감염,기타 확진자 접촉,미정,1,중랑구20.12.31.
4,20.12.31.,강북구,국내감염,감염경로 조사중,미정,1,강북구20.12.31.
...,...,...,...,...,...,...,...
19358,20.1.31.,성북구,국내감염,#5 접촉,퇴원,1,성북구20.1.31.
19359,20.1.30.,마포구,중국 우한시,해외 접촉,퇴원,1,마포구20.1.30.
19360,20.1.30.,종로구,국내감염,#3 접촉,퇴원,1,종로구20.1.30.
19361,20.1.30.,중랑구,중국 우한시,해외 접촉,퇴원,1,중랑구20.1.30.


In [52]:
# 지역(구)별로 통합
seoul_corona3 = seoul_corona2.groupby("지역")
seoul_corona3.head(10)

,확진일,지역,여행력,접촉력,상태,확진자 수
0,20.12.31.,강북구,국내감염,감염경로 조사중,미정,1
1,20.12.31.,양천구,국내감염,기타 확진자 접촉,미정,1
2,20.12.31.,도봉구,국내감염,기타 확진자 접촉,미정,1
3,20.12.31.,중랑구,국내감염,기타 확진자 접촉,미정,1
4,20.12.31.,강북구,국내감염,감염경로 조사중,미정,1
...,...,...,...,...,...,...
1086,20.12.29.,용산구,국내감염,감염경로 조사중,미정,1
1105,20.12.29.,용산구,국내감염,기타 확진자 접촉,미정,1
1631,20.12.28.,성동구,국내감염,기타 확진자 접촉,미정,1
1632,20.12.27.,성동구,국내감염,기타 확진자 접촉,미정,1


In [50]:
# 날짜별로 통합
seoul_corona3 = seoul_corona3.groupby("확진일")
seoul_corona3.head(20)

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'

In [ ]:
# 구별 컬럼을 인덱스로 변경
seoul_corona3_result.set_index("지역", inplace=True)
data_result

In [29]:
# 확진일 기준으로, 지역이름 가나다순으로 재정렬
#seoul_corona3 = seoul_corona3.sort_values(by="지역", ascending=False)
#seoul_corona3

AttributeError: 'DataFrameGroupBy' object has no attribute 'sort_values'

##### 2-2. 날짜별로 구분 
+ row data 날짜 기준으로 구분(index = 확정일)
+ 날짜별 확진자 수 막대그래프
+ 시계열 분석으로 내년 2사분위까지 예상해보기

In [30]:
#확진일 별로 묶기
seoul_corona4 = seoul_corona3.groupby("확진일")
seoul_corona4.head(10)

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'

In [36]:
# 지역 컬럼을 인덱스로 바꿈
seoul_corona2 = seoul_corona2.set_index(["지역"])  
#print(seoul_corona2, drop=False)
seoul_corona2

,확진일,상태,여행력,접촉력
지역,,,,
강북구,20.12.31.,미정,국내감염,감염경로 조사중
양천구,20.12.31.,미정,국내감염,기타 확진자 접촉
도봉구,20.12.31.,미정,국내감염,기타 확진자 접촉
중랑구,20.12.31.,미정,국내감염,기타 확진자 접촉
강북구,20.12.31.,미정,국내감염,감염경로 조사중
...,...,...,...,...
성북구,20.1.31.,퇴원,국내감염,#5 접촉
마포구,20.1.30.,퇴원,중국 우한시,해외 접촉
종로구,20.1.30.,퇴원,국내감염,#3 접촉


In [38]:
print(type(seoul_corona2["확진일"]))

<class 'pandas.core.series.Series'>


In [39]:
#####  Checkpoint  #####
seoul_corona2.to_csv("data/seoul/seoul_corona2.csv", encoding="utf-8")

### 3. 데이터2 전처리

In [36]:
# 데이터2 : 
# 서울시 코로나 사망자 데이터(보건복지부_코로나19 연령별·성별감염_현황)

In [7]:
import urllib as url
import urllib.parse
import requests

In [8]:
from urllib.request import urlopen, Request
from urllib.parse import urlencode, quote_plus

In [22]:
# 데이터 호출(API)
key = "jyYQv4gijATld1TBLxc0v5MYf1Yo%2BdyMMC0TvPd3joZAD14maSu3SGjyv%2BpYn%2BDXijtucBM33Gd3Cvz1sYLUCQ%3D%3D"
api = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson?serviceKey={}".format(key)

queryParams = '?' + urlencode({ quote_plus('ServiceKey') : 'key', 
                               quote_plus('pageNo') : '1', 
                               quote_plus('numOfRows') : '10', 
                               quote_plus('startCreateDt') : '20200310', 
                               quote_plus('endCreateDt') : '20200414'})

In [17]:
# 사망자 데이터 보기
value = {"DEATH":"0"}

params = urllib.parse.urlencode(value)
print(params)

DEATH=0


In [21]:
url = api + "?" + param
print(url)

data = urlopen(url).read()
print(data)

#data = data.decode("utf-8")
#print(data)

http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson?serviceKey={}?DEATH=0
b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>'
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>


In [24]:
##request = Request(url + queryParams)
##request.get_method = lambda: "GET"
##response_body = urlopen(request).read()
##print(response_body)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>'


In [44]:
# 파일 읽어오기
content = requests.get(url).content
print(type(content))

<class 'bytes'>


### 3. 데이터3 전처리

In [2]:
# 데이터1 : 12월 31일 데이터
seoul_corona3 = pd.read_csv("data/seoul/서울시 주민등록인구 (동별)통계.txt", sep="\t, encoding="euc-kr", thousands=",")
seoul_corona3.head(10)

SyntaxError: invalid syntax (<ipython-input-2-2639cf58b5c2>, line 2)

In [38]:
data = pd.read_csv('data/seoul/서울시 주민등록인구 (동별)통계.txt', sep = "\t", , engine='python', encoding = "utf-8")
data.head()

SyntaxError: invalid syntax (<ipython-input-38-0a2508d19f95>, line 1)

### 4. 시각화

In [ ]:
import json
import folium
import googlemaps

In [ ]:
geo_path = "data/seoul/skorea_municipalities_geo_simple_seoul.json"
geo_str = json.load(open(geo_path, encoding="utf-8"))  # 한글 인코딩

map = folium.Map(location=[37.501826, 127.039870], zoom_start=10)

map.choropleth(geo_data=geo_str, data=gu_data, 
               columns=[gu_data.index, "가격"],
               key_on="feature.id", fill_color="YlGn")
map

### 5. 결론